In [ ]:
import pip
pip.main(['install', 'pandas'])
pip.main(['install', 'matplotlib'])
pip.main(['install', 'seaborn'])
pip.main(['install', 'scikit-learn'])

CHARGEMENT DES BIBLIOTHEQUES

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


CHARGEMENT DU JEU DE DONNEES

In [3]:
dataset_path = "C:\\Users\\belga\\OneDrive\\Bureau\\EMBA Big Data\\Module Datascience\\Projet Data Science S2 2024\\CLIENT.csv"
CLIENT = pd.read_csv(dataset_path, sep='|', low_memory=False)
print(CLIENT.shape)
print(list(CLIENT.columns))

(845876, 10)
['IDCLIENT', 'CIVILITE', 'DATENAISSANCE', 'MAGASIN', 'DATEDEBUTADHESION', 'DATEREADHESION', 'DATEFINADHESION', 'VIP', 'CODEINSEE', 'PAYS']


La ligne de code ci-dessous nous permet de connaître le détail des colonnes ainsi que le type de données constituant notre base (numérique ou caractères).

In [4]:
CLIENT

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,VIP,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005 0:00:00,4/04/2017 0:00:00,30/04/2018 0:00:00,0,74224,FR
1,156023.0,MONSIEUR,7/08/1966 0:00:00,VLG,14/05/2005 0:00:00,21/06/2014 0:00:00,30/06/2016 0:00:00,0,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005 0:00:00,12/10/2017 0:00:00,31/10/2018 0:00:00,0,63070,FR
3,156190.0,MADAME,24/07/1976 0:00:00,SUR,28/08/2006 0:00:00,24/05/2015 0:00:00,31/05/2016 0:00:00,0,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006 0:00:00,18/12/2016 0:00:00,31/12/2017 0:00:00,0,31555,FR
...,...,...,...,...,...,...,...,...,...,...
845871,1970111.0,MONSIEUR,NaN,IAB,14/04/2011 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,0,38053,FR
845872,2297275.0,MONSIEUR,6/02/1988 0:00:00,SEM,7/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,0,57067,FR
845873,2305548.0,MONSIEUR,NaN,PRI,22/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,0,38318,FR
845874,84175.0,MADAME,7/04/1950 0:00:00,GAI,29/10/2002 0:00:00,7/01/2018 0:00:00,31/01/2019 0:00:00,0,74082,FR


In [5]:
CLIENT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845876 entries, 0 to 845875
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   IDCLIENT           845876 non-null  float64
 1   CIVILITE           845876 non-null  object 
 2   DATENAISSANCE      507902 non-null  object 
 3   MAGASIN            845876 non-null  object 
 4   DATEDEBUTADHESION  845876 non-null  object 
 5   DATEREADHESION     507816 non-null  object 
 6   DATEFINADHESION    845876 non-null  object 
 7   VIP                845876 non-null  int64  
 8   CODEINSEE          818455 non-null  object 
 9   PAYS               845873 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 64.5+ MB


Nous allons procéder à la suppression des clients VIP, qui ont un comportement atypique qui viendra biaiser nos résultats.

In [6]:
CLIENT.head()

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,VIP,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005 0:00:00,4/04/2017 0:00:00,30/04/2018 0:00:00,0,74224,FR
1,156023.0,MONSIEUR,7/08/1966 0:00:00,VLG,14/05/2005 0:00:00,21/06/2014 0:00:00,30/06/2016 0:00:00,0,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005 0:00:00,12/10/2017 0:00:00,31/10/2018 0:00:00,0,63070,FR
3,156190.0,MADAME,24/07/1976 0:00:00,SUR,28/08/2006 0:00:00,24/05/2015 0:00:00,31/05/2016 0:00:00,0,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006 0:00:00,18/12/2016 0:00:00,31/12/2017 0:00:00,0,31555,FR


In [7]:
# Supprimer les lignes où la valeur de la colonne 'VIP' est égale à 1
CLIENT = CLIENT[CLIENT['VIP'] != 1]

# Vérifier la nouvelle forme du DataFrame
print(CLIENT.shape)

(735425, 10)


In [8]:
CLIENT = CLIENT.drop("VIP",axis=1)

Vérification du traitement.

In [9]:
CLIENT

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005 0:00:00,4/04/2017 0:00:00,30/04/2018 0:00:00,74224,FR
1,156023.0,MONSIEUR,7/08/1966 0:00:00,VLG,14/05/2005 0:00:00,21/06/2014 0:00:00,30/06/2016 0:00:00,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005 0:00:00,12/10/2017 0:00:00,31/10/2018 0:00:00,63070,FR
3,156190.0,MADAME,24/07/1976 0:00:00,SUR,28/08/2006 0:00:00,24/05/2015 0:00:00,31/05/2016 0:00:00,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006 0:00:00,18/12/2016 0:00:00,31/12/2017 0:00:00,31555,FR
...,...,...,...,...,...,...,...,...,...
845871,1970111.0,MONSIEUR,NaN,IAB,14/04/2011 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,38053,FR
845872,2297275.0,MONSIEUR,6/02/1988 0:00:00,SEM,7/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,57067,FR
845873,2305548.0,MONSIEUR,NaN,PRI,22/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,38318,FR
845874,84175.0,MADAME,7/04/1950 0:00:00,GAI,29/10/2002 0:00:00,7/01/2018 0:00:00,31/01/2019 0:00:00,74082,FR


La ligne de code ci-dessous nous permet de formater le format de la date.

In [10]:
# Adapter le format de date pour les colonnes spécifiées
date_columns = ['DATENAISSANCE', 'DATEDEBUTADHESION', 'DATEREADHESION', 'DATEFINADHESION']
for col in date_columns:
    CLIENT[col] = pd.to_datetime(CLIENT[col], format='%d/%m/%Y %H:%M:%S',errors = 'coerce').dt.strftime('%d/%m/%Y')

Vérification du traitement.

In [11]:
CLIENT

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR
1,156023.0,MONSIEUR,07/08/1966,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR
3,156190.0,MADAME,24/07/1976,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR
...,...,...,...,...,...,...,...,...,...
845871,1970111.0,MONSIEUR,NaN,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR
845872,2297275.0,MONSIEUR,06/02/1988,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR
845873,2305548.0,MONSIEUR,NaN,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR
845874,84175.0,MADAME,07/04/1950,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR


Cette ligne de code nous permet d'identifier le nombre d'occurence de chaque civilité, pour identifier les éventuels regroupements à effectuer. 

In [12]:
#modalités variables:
CLIENT['CIVILITE'].value_counts(dropna = False)

CIVILITE
MADAME      393703
MONSIEUR    337360
Mr            1431
Mme           1284
madame         976
monsieur       671
Name: count, dtype: int64

Il convient de procéder aux regroupements ci-dessous.

In [13]:
#regroupement des modalités:
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='MADAME', 'Madame', CLIENT['CIVILITE']) 
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='Mme', 'Madame', CLIENT['CIVILITE'])
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='madame', 'Madame', CLIENT['CIVILITE']) 
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='MONSIEUR', 'Monsieur', CLIENT['CIVILITE'])
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='Mr', 'Monsieur', CLIENT['CIVILITE'])
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='monsieur', 'Monsieur', CLIENT['CIVILITE'])

Vérification du traitement.

In [14]:
CLIENT['CIVILITE'].value_counts(dropna = False)

CIVILITE
Madame      395963
Monsieur    339462
Name: count, dtype: int64

In [15]:
# Mapping de la colonne CIVILITE pour créer la nouvelle colonne SEXE
CLIENT['SEXE'] = CLIENT['CIVILITE'].map({'Monsieur': 'HOMME', 'Madame': 'FEMME'})

print(CLIENT)


         IDCLIENT  CIVILITE DATENAISSANCE MAGASIN DATEDEBUTADHESION  \
0        155891.0    Madame           NaN     VLG        13/05/2005   
1        156023.0  Monsieur    07/08/1966     VLG        14/05/2005   
2        124203.0    Madame           NaN     BEA        16/03/2005   
3        156190.0    Madame    24/07/1976     SUR        28/08/2006   
4        144444.0  Monsieur           NaN     LAB        29/04/2006   
...           ...       ...           ...     ...               ...   
845871  1970111.0  Monsieur           NaN     IAB        14/04/2011   
845872  2297275.0  Monsieur    06/02/1988     SEM        07/03/2013   
845873  2305548.0  Monsieur           NaN     PRI        22/03/2013   
845874    84175.0    Madame    07/04/1950     GAI        29/10/2002   
845875  1280004.0  Monsieur           NaN     SGL        23/06/2007   

       DATEREADHESION DATEFINADHESION CODEINSEE PAYS   SEXE  
0          04/04/2017      30/04/2018     74224   FR  FEMME  
1          21/06/2014  

In [16]:
#Grouper les Clients par Sexe
clients_par_sexe = CLIENT.groupby('SEXE')['IDCLIENT'].nunique().reset_index(name='NbClients')

print(clients_par_sexe)

    SEXE  NbClients
0  FEMME     395963
1  HOMME     339462


In [17]:
# Supprimer l'ancienne colonne CIVILITE
CLIENT.drop(columns=['CIVILITE'], inplace=True)

In [18]:
CLIENT

,IDCLIENT,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS,SEXE
0,155891.0,NaN,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR,FEMME
1,156023.0,07/08/1966,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR,HOMME
2,124203.0,NaN,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR,FEMME
3,156190.0,24/07/1976,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR,FEMME
4,144444.0,NaN,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR,HOMME
...,...,...,...,...,...,...,...,...,...
845871,1970111.0,NaN,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR,HOMME
845872,2297275.0,06/02/1988,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR,HOMME
845873,2305548.0,NaN,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR,HOMME
845874,84175.0,07/04/1950,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR,FEMME


Cette ligne de code nous permet d'identifier le nombre d'occurence de chaque magasin, pour identifier les éventuels regroupements à effectuer.

In [19]:
CLIENT['MAGASIN'].value_counts(dropna = False)

MAGASIN
PRI    23963
SEY    23572
VIB    22223
SUR    20923
MOU    20887
       ...  
VIV     3187
SAL     2836
STR     2667
EST     1145
BAR     1039
Name: count, Length: 67, dtype: int64

Les données à traiter concernant la France, la série de code ci-dessous nous permet de ne conserver que ce pays.

In [20]:
# Filtrer les lignes pour ne garder que celles avec "France" dans la colonne "Pays"
CLIENT = CLIENT[CLIENT['PAYS'] == 'FR']

# Réindexer le DataFrame après avoir filtré les lignes
CLIENT.reset_index(drop=True, inplace=True)

In [21]:
CLIENT['PAYS'].value_counts(dropna = False)

PAYS
FR    720090
Name: count, dtype: int64

In [22]:
CLIENT.shape

(720090, 9)

La serie de codes ci-dessous nous permet de traiter la date de naissance des clients afin d'en déterminer l'âge puis des tranches d'âges.

In [23]:
import pandas as pd
from datetime import datetime# Create the DataFrame
# Convertir la colonne 'DATENAISSANCE' au format datetime
CLIENT['DATENAISSANCE'] = pd.to_datetime(CLIENT['DATENAISSANCE'], dayfirst=True, errors='coerce')

# Définir la date de référence pour le calcul de l'âge
reference_date = datetime(2018, 1, 1)

# Calculer l'âge des clients basé sur la date de référence
CLIENT['AGE'] = reference_date.year - CLIENT['DATENAISSANCE'].dt.year

# Ajuster l'âge si l'anniversaire n'est pas encore passé cette année-là
past_birthdays = (reference_date.month > CLIENT['DATENAISSANCE'].dt.month) | \
                 ((reference_date.month == CLIENT['DATENAISSANCE'].dt.month) & (reference_date.day >= CLIENT['DATENAISSANCE'].dt.day))

# Appliquer l'ajustement pertinent
CLIENT.loc[~past_birthdays, 'AGE'] -= 1

# Définir les intervalles d'âge et les étiquettes correspondantes
age_bins = [18, 30, 45, 60, 100]
age_labels = ['AGE DE 18-30', 'AGE DE 31-45', 'AGE DE 46-60', 'AGE DE 61-100']

# Catégoriser les âges dans les intervalles définis
CLIENT['AGE_GROUP'] = pd.cut(CLIENT['AGE'], bins=age_bins, labels=age_labels, include_lowest=True)

# Afficher le DataFrame avec les nouvelles colonnes
print(CLIENT)

         IDCLIENT DATENAISSANCE MAGASIN DATEDEBUTADHESION DATEREADHESION  \
0        155891.0           NaT     VLG        13/05/2005     04/04/2017   
1        156023.0    1966-08-07     VLG        14/05/2005     21/06/2014   
2        124203.0           NaT     BEA        16/03/2005     12/10/2017   
3        156190.0    1976-07-24     SUR        28/08/2006     24/05/2015   
4        144444.0           NaT     LAB        29/04/2006     18/12/2016   
...           ...           ...     ...               ...            ...   
720085  1970111.0           NaT     IAB        14/04/2011     06/01/2018   
720086  2297275.0    1988-02-06     SEM        07/03/2013     06/01/2018   
720087  2305548.0           NaT     PRI        22/03/2013     06/01/2018   
720088    84175.0    1950-04-07     GAI        29/10/2002     07/01/2018   
720089  1280004.0           NaT     SGL        23/06/2007     07/01/2018   

       DATEFINADHESION CODEINSEE PAYS   SEXE   AGE      AGE_GROUP  
0           30/04/2

C:\Users\belga\AppData\Local\Temp\ipykernel_7028\3627133065.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLIENT['DATENAISSANCE'] = pd.to_datetime(CLIENT['DATENAISSANCE'], dayfirst=True, errors='coerce')
C:\Users\belga\AppData\Local\Temp\ipykernel_7028\3627133065.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLIENT['AGE'] = reference_date.year - CLIENT['DATENAISSANCE'].dt.year
C:\Users\belga\AppData\Local\Temp\ipykernel_7028\3627133065.py:24: SettingWithCopyWarning: 
A value is trying to be set

Vérification du traitement.

In [24]:
CLIENT['AGE_GROUP'].value_counts(dropna = False)

AGE_GROUP
NaN              288110
AGE DE 46-60     137070
AGE DE 31-45     132554
AGE DE 61-100    111017
AGE DE 18-30      51339
Name: count, dtype: int64

In [25]:
CLIENT.shape

(720090, 11)

Après ce traitement, il convient de renommer la variable concernée.

In [26]:
# Renommer la colonne 'DATENAISSANCE' en 'AGE'
CLIENT = CLIENT.rename(columns={'DATENAISSANCE': 'AGE'})

Vérification du traitement.

In [27]:
CLIENT

,IDCLIENT,AGE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS,SEXE,AGE,AGE_GROUP
0,155891.0,NaT,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR,FEMME,NaN,NaN
1,156023.0,1966-08-07,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR,HOMME,51.0,AGE DE 46-60
2,124203.0,NaT,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR,FEMME,NaN,NaN
3,156190.0,1976-07-24,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR,FEMME,41.0,AGE DE 31-45
4,144444.0,NaT,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR,HOMME,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
720085,1970111.0,NaT,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR,HOMME,NaN,NaN
720086,2297275.0,1988-02-06,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR,HOMME,29.0,AGE DE 18-30
720087,2305548.0,NaT,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR,HOMME,NaN,NaN
720088,84175.0,1950-04-07,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR,FEMME,67.0,AGE DE 61-100


Afin d'exploiter de façon optimale les informations, nous allons convetir les l'ID TICKET, l'âge et les tranches d'âges en caractère. Nous n'avons pas nécessité à conserver ces valeurs en numérique.

In [28]:
CLIENT['IDCLIENT'] = CLIENT['IDCLIENT'].astype(str)
CLIENT['AGE'] = CLIENT['AGE'].astype(str)
CLIENT['AGE_GROUP'] = CLIENT['AGE_GROUP'].astype(str)

Vérification du traitement.

In [29]:
CLIENT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720090 entries, 0 to 720089
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   IDCLIENT           720090 non-null  object
 1   AGE                720090 non-null  object
 2   MAGASIN            720090 non-null  object
 3   DATEDEBUTADHESION  720090 non-null  object
 4   DATEREADHESION     410990 non-null  object
 5   DATEFINADHESION    720090 non-null  object
 6   CODEINSEE          709392 non-null  object
 7   PAYS               720090 non-null  object
 8   SEXE               720090 non-null  object
 9   AGE                720090 non-null  object
 10  AGE_GROUP          720090 non-null  object
dtypes: object(11)
memory usage: 60.4+ MB


La ligne de code ci-dessous nous permet d'identifier les valeurs en doublons qui conviendra de traiter.

In [30]:
print(CLIENT.duplicated().sum())
CLIENT = CLIENT.drop_duplicates()

0


Nous procédons également à la vérification de présence ou non de valeurs manquantes.

In [31]:
pd.DataFrame(index=CLIENT.columns, columns=['%_missing_values'], data= (CLIENT.isna().sum().values / len(CLIENT)*100))

,%_missing_values
IDCLIENT,0.000000
AGE,0.000000
MAGASIN,0.000000
DATEDEBUTADHESION,0.000000
DATEREADHESION,42.925190
DATEFINADHESION,0.000000
CODEINSEE,1.485648
PAYS,0.000000
SEXE,0.000000
AGE,0.000000


La série de codes ci-dessous nous permet de traiter les valeurs manquantes détéctées, sur les variables concernées.

In [32]:
CLIENT['CODEINSEE'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
720085    False
720086    False
720087    False
720088    False
720089    False
Name: CODEINSEE, Length: 720090, dtype: bool

In [33]:
CLIENT.dropna(subset=['CODEINSEE'], inplace=True)

In [34]:
CLIENT['DATEREADHESION'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
720085    False
720086    False
720087    False
720088    False
720089    False
Name: DATEREADHESION, Length: 709392, dtype: bool

In [35]:
CLIENT.dropna(subset=['DATEREADHESION'], inplace=True)

In [36]:
CLIENT['AGE_GROUP'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
720085    False
720086    False
720087    False
720088    False
720089    False
Name: AGE_GROUP, Length: 407389, dtype: bool

In [37]:
CLIENT['AGE_GROUP'].value_counts()

AGE_GROUP
nan              202154
AGE DE 46-60      73722
AGE DE 61-100     66788
AGE DE 31-45      53674
AGE DE 18-30      11051
Name: count, dtype: int64

In [38]:
CLIENT['AGE_GROUP'] = CLIENT['AGE_GROUP'].replace(np.nan, "Inconnue")


In [39]:
# Utilisation de `map` pour remplacer 'nan' par 'Inconnue'
CLIENT['AGE_GROUP'] = CLIENT['AGE_GROUP'].map(lambda x: 'Inconnue' if x == 'nan' else x)

# Affichage du résultat
print(CLIENT)

         IDCLIENT         AGE MAGASIN DATEDEBUTADHESION DATEREADHESION  \
0        155891.0         NaT     VLG        13/05/2005     04/04/2017   
1        156023.0  1966-08-07     VLG        14/05/2005     21/06/2014   
2        124203.0         NaT     BEA        16/03/2005     12/10/2017   
3        156190.0  1976-07-24     SUR        28/08/2006     24/05/2015   
4        144444.0         NaT     LAB        29/04/2006     18/12/2016   
...           ...         ...     ...               ...            ...   
720085  1970111.0         NaT     IAB        14/04/2011     06/01/2018   
720086  2297275.0  1988-02-06     SEM        07/03/2013     06/01/2018   
720087  2305548.0         NaT     PRI        22/03/2013     06/01/2018   
720088    84175.0  1950-04-07     GAI        29/10/2002     07/01/2018   
720089  1280004.0         NaT     SGL        23/06/2007     07/01/2018   

       DATEFINADHESION CODEINSEE PAYS   SEXE   AGE      AGE_GROUP  
0           30/04/2018     74224   FR  FEMM

In [40]:
CLIENT['AGE_GROUP'].value_counts()

AGE_GROUP
Inconnue         202154
AGE DE 46-60      73722
AGE DE 61-100     66788
AGE DE 31-45      53674
AGE DE 18-30      11051
Name: count, dtype: int64

In [41]:
CLIENT.dropna(subset=['AGE_GROUP'], inplace=True)

Vérification du taitement.

In [42]:
CLIENT

,IDCLIENT,AGE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS,SEXE,AGE,AGE_GROUP
0,155891.0,NaT,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR,FEMME,nan,Inconnue
1,156023.0,1966-08-07,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR,HOMME,51.0,AGE DE 46-60
2,124203.0,NaT,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR,FEMME,nan,Inconnue
3,156190.0,1976-07-24,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR,FEMME,41.0,AGE DE 31-45
4,144444.0,NaT,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR,HOMME,nan,Inconnue
...,...,...,...,...,...,...,...,...,...,...,...
720085,1970111.0,NaT,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR,HOMME,nan,Inconnue
720086,2297275.0,1988-02-06,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR,HOMME,29.0,AGE DE 18-30
720087,2305548.0,NaT,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR,HOMME,nan,Inconnue
720088,84175.0,1950-04-07,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR,FEMME,67.0,AGE DE 61-100


In [43]:
pd.DataFrame(index=CLIENT.columns, columns=['%_missing_values'], data= (CLIENT.isna().sum().values / len(CLIENT)*100))

,%_missing_values
IDCLIENT,0.0
AGE,0.0
MAGASIN,0.0
DATEDEBUTADHESION,0.0
DATEREADHESION,0.0
DATEFINADHESION,0.0
CODEINSEE,0.0
PAYS,0.0
SEXE,0.0
AGE,0.0


Le traitement des valeurs aberrantes ne concernant que les variables numériques, nous n'allons pas procéder à ce traitement car aucune de nos variables n'est concernées sur ce dataframe.

Suppression des colonnes dont les informations nous serons pas nécessairement utiles dans le cadre de notre projet.

In [44]:
CLIENT.drop(columns=['PAYS'], inplace=True)

In [45]:
CLIENT.drop(columns=['AGE'], inplace=True)

In [46]:
CLIENT.drop(columns=['CODEINSEE'], inplace=True)

In [47]:
CLIENT

,IDCLIENT,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,SEXE,AGE_GROUP
0,155891.0,VLG,13/05/2005,04/04/2017,30/04/2018,FEMME,Inconnue
1,156023.0,VLG,14/05/2005,21/06/2014,30/06/2016,HOMME,AGE DE 46-60
2,124203.0,BEA,16/03/2005,12/10/2017,31/10/2018,FEMME,Inconnue
3,156190.0,SUR,28/08/2006,24/05/2015,31/05/2016,FEMME,AGE DE 31-45
4,144444.0,LAB,29/04/2006,18/12/2016,31/12/2017,HOMME,Inconnue
...,...,...,...,...,...,...,...
720085,1970111.0,IAB,14/04/2011,06/01/2018,31/01/2019,HOMME,Inconnue
720086,2297275.0,SEM,07/03/2013,06/01/2018,31/01/2019,HOMME,AGE DE 18-30
720087,2305548.0,PRI,22/03/2013,06/01/2018,31/01/2019,HOMME,Inconnue
720088,84175.0,GAI,29/10/2002,07/01/2018,31/01/2019,FEMME,AGE DE 61-100


La série de codes ci-dessous nous permet de modifier le format des dates de début et de fin d'adhésion afin de calculer l'ancienneté des clients.

In [48]:
import datetime
from datetime import timedelta
# Convertir les colonnes en type datetime
CLIENT['DATEDEBUTADHESION'] = pd.to_datetime(CLIENT['DATEDEBUTADHESION'], dayfirst=True)
CLIENT['DATEFINADHESION'] = pd.to_datetime(CLIENT['DATEFINADHESION'], dayfirst=True)

# Calculer l'ancienneté en années
CLIENT['ANCIENNETE_ADHESION'] = (CLIENT['DATEFINADHESION'] - CLIENT['DATEDEBUTADHESION']) / timedelta(days=365.25)

# Définir les bins en années
bins = [0, 1, 2, 3, 5, 8, float('inf')]
labels = ['0-1 an', '1-2 ans', '2-3 ans', '3-5 ans', '5-8 ans','8 ans+']

# Créer une nouvelle colonne 'ANCIENNETE_ADHESION_CAT' pour les catégories basées sur les bins d'années
CLIENT['ANCIENNETE_ADHESION_CAT'] = pd.cut(CLIENT['ANCIENNETE_ADHESION'], bins=bins, labels=labels, right=False)

# Afficher le DataFrame mis à jour
print(CLIENT)

# Voir la description des bins
print(CLIENT['ANCIENNETE_ADHESION_CAT'].value_counts())

         IDCLIENT MAGASIN DATEDEBUTADHESION DATEREADHESION DATEFINADHESION  \
0        155891.0     VLG        2005-05-13     04/04/2017      2018-04-30   
1        156023.0     VLG        2005-05-14     21/06/2014      2016-06-30   
2        124203.0     BEA        2005-03-16     12/10/2017      2018-10-31   
3        156190.0     SUR        2006-08-28     24/05/2015      2016-05-31   
4        144444.0     LAB        2006-04-29     18/12/2016      2017-12-31   
...           ...     ...               ...            ...             ...   
720085  1970111.0     IAB        2011-04-14     06/01/2018      2019-01-31   
720086  2297275.0     SEM        2013-03-07     06/01/2018      2019-01-31   
720087  2305548.0     PRI        2013-03-22     06/01/2018      2019-01-31   
720088    84175.0     GAI        2002-10-29     07/01/2018      2019-01-31   
720089  1280004.0     SGL        2007-06-23     07/01/2018      2019-01-31   

         SEXE      AGE_GROUP  ANCIENNETE_ADHESION ANCIENNETE_AD

In [49]:
CLIENT

,IDCLIENT,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,SEXE,AGE_GROUP,ANCIENNETE_ADHESION,ANCIENNETE_ADHESION_CAT
0,155891.0,VLG,2005-05-13,04/04/2017,2018-04-30,FEMME,Inconnue,12.963723,8 ans+
1,156023.0,VLG,2005-05-14,21/06/2014,2016-06-30,HOMME,AGE DE 46-60,11.129363,8 ans+
2,124203.0,BEA,2005-03-16,12/10/2017,2018-10-31,FEMME,Inconnue,13.626283,8 ans+
3,156190.0,SUR,2006-08-28,24/05/2015,2016-05-31,FEMME,AGE DE 31-45,9.757700,8 ans+
4,144444.0,LAB,2006-04-29,18/12/2016,2017-12-31,HOMME,Inconnue,11.674196,8 ans+
...,...,...,...,...,...,...,...,...,...
720085,1970111.0,IAB,2011-04-14,06/01/2018,2019-01-31,HOMME,Inconnue,7.800137,5-8 ans
720086,2297275.0,SEM,2013-03-07,06/01/2018,2019-01-31,HOMME,AGE DE 18-30,5.902806,5-8 ans
720087,2305548.0,PRI,2013-03-22,06/01/2018,2019-01-31,HOMME,Inconnue,5.861739,5-8 ans
720088,84175.0,GAI,2002-10-29,07/01/2018,2019-01-31,FEMME,AGE DE 61-100,16.257358,8 ans+


Ci-dessous, nous procédons à la suppression des dates de début et de fin d'adhésion car nous n'en avons plus besoin.

In [50]:
CLIENT.drop(columns=['DATEDEBUTADHESION'], inplace=True)

In [51]:
CLIENT.drop(columns=['DATEFINADHESION'], inplace=True)

Vérification des traitements.

In [52]:
CLIENT

,IDCLIENT,MAGASIN,DATEREADHESION,SEXE,AGE_GROUP,ANCIENNETE_ADHESION,ANCIENNETE_ADHESION_CAT
0,155891.0,VLG,04/04/2017,FEMME,Inconnue,12.963723,8 ans+
1,156023.0,VLG,21/06/2014,HOMME,AGE DE 46-60,11.129363,8 ans+
2,124203.0,BEA,12/10/2017,FEMME,Inconnue,13.626283,8 ans+
3,156190.0,SUR,24/05/2015,FEMME,AGE DE 31-45,9.757700,8 ans+
4,144444.0,LAB,18/12/2016,HOMME,Inconnue,11.674196,8 ans+
...,...,...,...,...,...,...,...
720085,1970111.0,IAB,06/01/2018,HOMME,Inconnue,7.800137,5-8 ans
720086,2297275.0,SEM,06/01/2018,HOMME,AGE DE 18-30,5.902806,5-8 ans
720087,2305548.0,PRI,06/01/2018,HOMME,Inconnue,5.861739,5-8 ans
720088,84175.0,GAI,07/01/2018,FEMME,AGE DE 61-100,16.257358,8 ans+


In [53]:
CLIENT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407389 entries, 0 to 720089
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   IDCLIENT                 407389 non-null  object  
 1   MAGASIN                  407389 non-null  object  
 2   DATEREADHESION           407389 non-null  object  
 3   SEXE                     407389 non-null  object  
 4   AGE_GROUP                407389 non-null  object  
 5   ANCIENNETE_ADHESION      407389 non-null  float64 
 6   ANCIENNETE_ADHESION_CAT  407389 non-null  category
dtypes: category(1), float64(1), object(5)
memory usage: 22.1+ MB


Le code ci-dessous va également nous permettre de modifier le format de la date de ré-adhésion.

In [54]:
# Convertir la colonne 'DATEREADHESION' en type datetime en spécifiant dayfirst=True
CLIENT['DATEREADHESION'] = pd.to_datetime(CLIENT['DATEREADHESION'], dayfirst=True)

# Filtrer pour ne garder que les adhésions de l'année 2017
CLIENT = CLIENT[CLIENT['DATEREADHESION'].dt.year == 2017]

# Vérifier la nouvelle forme du DataFrame
print(CLIENT.shape)

# Afficher le DataFrame pour vérifier les résultats
print(CLIENT)

(203760, 7)
         IDCLIENT MAGASIN DATEREADHESION   SEXE      AGE_GROUP  \
0        155891.0     VLG     2017-04-04  FEMME       Inconnue   
2        124203.0     BEA     2017-10-12  FEMME       Inconnue   
5        104768.0     SMR     2017-11-05  FEMME       Inconnue   
8        104550.0     OBE     2017-10-19  FEMME  AGE DE 61-100   
9        104601.0     OBE     2017-09-01  FEMME   AGE DE 31-45   
...           ...     ...            ...    ...            ...   
719975   336771.0     BEA     2017-12-30  FEMME       Inconnue   
719976  1709516.0     VIB     2017-12-30  HOMME       Inconnue   
719977  1866695.0     MAN     2017-12-30  FEMME  AGE DE 61-100   
719978  2098666.0     CLI     2017-12-30  HOMME       Inconnue   
720049   402282.0     VIB     2017-12-31  HOMME   AGE DE 46-60   

        ANCIENNETE_ADHESION ANCIENNETE_ADHESION_CAT  
0                 12.963723                  8 ans+  
2                 13.626283                  8 ans+  
5                 13.719370      

In [55]:
# Supposons que 'date_max' est initialisé quelque part avant
date_ref = pd.to_datetime('31/12/2017', dayfirst=True)
# Si CLIENT est déjà filtré, assurez-vous de travailler sur une copie propre de ce DataFrame
CLIENT = CLIENT.copy()

# Calculer l'ancienneté en jours
CLIENT.loc[:, 'Anciennete'] = (date_ref - CLIENT['DATEREADHESION']).dt.days

# Remplace les valeurs NaN par 0 jours, et mettre cette colonne à un type entier.
CLIENT.loc[:, 'Anciennete'] = CLIENT['Anciennete'].fillna(0).astype(int)

# Vérifier la nouvelle forme du DataFrame
print(CLIENT.shape)

# Afficher le DataFrame
print(CLIENT)

(203760, 8)
         IDCLIENT MAGASIN DATEREADHESION   SEXE      AGE_GROUP  \
0        155891.0     VLG     2017-04-04  FEMME       Inconnue   
2        124203.0     BEA     2017-10-12  FEMME       Inconnue   
5        104768.0     SMR     2017-11-05  FEMME       Inconnue   
8        104550.0     OBE     2017-10-19  FEMME  AGE DE 61-100   
9        104601.0     OBE     2017-09-01  FEMME   AGE DE 31-45   
...           ...     ...            ...    ...            ...   
719975   336771.0     BEA     2017-12-30  FEMME       Inconnue   
719976  1709516.0     VIB     2017-12-30  HOMME       Inconnue   
719977  1866695.0     MAN     2017-12-30  FEMME  AGE DE 61-100   
719978  2098666.0     CLI     2017-12-30  HOMME       Inconnue   
720049   402282.0     VIB     2017-12-31  HOMME   AGE DE 46-60   

        ANCIENNETE_ADHESION ANCIENNETE_ADHESION_CAT  Anciennete  
0                 12.963723                  8 ans+         271  
2                 13.626283                  8 ans+          80

In [56]:
# Convertir l'ancienneté en mois (approximativement, en utilisant 30.44 jours par mois)
CLIENT['Anciennete_months'] = CLIENT['Anciennete'] / 30.44

# Définir les bins en mois (0-6 mois, 6-12 mois, 12-24 mois, 24-36 mois, 36+ mois)
bins = [0, 4, 6, 8, 10, np.inf]
labels = ['0-4 mois', '4-6 mois', '6-8 mois', '8-10 mois', '10+ mois']

# Discrétiser en utilisant les bins
CLIENT['ANCIENNETE_READ'] = pd.cut(CLIENT['Anciennete_months'], bins=bins, labels=labels, right=False)

# Afficher le DataFrame filtré et avec l'ancienneté
print(CLIENT)

         IDCLIENT MAGASIN DATEREADHESION   SEXE      AGE_GROUP  \
0        155891.0     VLG     2017-04-04  FEMME       Inconnue   
2        124203.0     BEA     2017-10-12  FEMME       Inconnue   
5        104768.0     SMR     2017-11-05  FEMME       Inconnue   
8        104550.0     OBE     2017-10-19  FEMME  AGE DE 61-100   
9        104601.0     OBE     2017-09-01  FEMME   AGE DE 31-45   
...           ...     ...            ...    ...            ...   
719975   336771.0     BEA     2017-12-30  FEMME       Inconnue   
719976  1709516.0     VIB     2017-12-30  HOMME       Inconnue   
719977  1866695.0     MAN     2017-12-30  FEMME  AGE DE 61-100   
719978  2098666.0     CLI     2017-12-30  HOMME       Inconnue   
720049   402282.0     VIB     2017-12-31  HOMME   AGE DE 46-60   

        ANCIENNETE_ADHESION ANCIENNETE_ADHESION_CAT  Anciennete  \
0                 12.963723                  8 ans+         271   
2                 13.626283                  8 ans+          80   
5     

In [57]:
CLIENT['ANCIENNETE_READ'].value_counts()

ANCIENNETE_READ
0-4 mois     68169
6-8 mois     48333
8-10 mois    48197
4-6 mois     23001
10+ mois     16060
Name: count, dtype: int64

In [58]:
CLIENT

,IDCLIENT,MAGASIN,DATEREADHESION,SEXE,AGE_GROUP,ANCIENNETE_ADHESION,ANCIENNETE_ADHESION_CAT,Anciennete,Anciennete_months,ANCIENNETE_READ
0,155891.0,VLG,2017-04-04,FEMME,Inconnue,12.963723,8 ans+,271,8.902760,8-10 mois
2,124203.0,BEA,2017-10-12,FEMME,Inconnue,13.626283,8 ans+,80,2.628121,0-4 mois
5,104768.0,SMR,2017-11-05,FEMME,Inconnue,13.719370,8 ans+,56,1.839685,0-4 mois
8,104550.0,OBE,2017-10-19,FEMME,AGE DE 61-100,13.544148,8 ans+,73,2.398160,0-4 mois
9,104601.0,OBE,2017-09-01,FEMME,AGE DE 31-45,13.456537,8 ans+,121,3.975033,0-4 mois
...,...,...,...,...,...,...,...,...,...,...
719975,336771.0,BEA,2017-12-30,FEMME,Inconnue,12.654346,8 ans+,1,0.032852,0-4 mois
719976,1709516.0,VIB,2017-12-30,HOMME,Inconnue,9.204654,8 ans+,1,0.032852,0-4 mois
719977,1866695.0,MAN,2017-12-30,FEMME,AGE DE 61-100,8.254620,8 ans+,1,0.032852,0-4 mois
719978,2098666.0,CLI,2017-12-30,HOMME,Inconnue,6.976044,5-8 ans,1,0.032852,0-4 mois


In [59]:
CLIENT.drop(columns=['Anciennete'], inplace=True)

In [60]:
CLIENT.drop(columns=['ANCIENNETE_ADHESION'], inplace=True)

In [61]:
CLIENT.drop(columns=['Anciennete_months'], inplace=True)

In [62]:
CLIENT.head()

,IDCLIENT,MAGASIN,DATEREADHESION,SEXE,AGE_GROUP,ANCIENNETE_ADHESION_CAT,ANCIENNETE_READ
0,155891.0,VLG,2017-04-04,FEMME,Inconnue,8 ans+,8-10 mois
2,124203.0,BEA,2017-10-12,FEMME,Inconnue,8 ans+,0-4 mois
5,104768.0,SMR,2017-11-05,FEMME,Inconnue,8 ans+,0-4 mois
8,104550.0,OBE,2017-10-19,FEMME,AGE DE 61-100,8 ans+,0-4 mois
9,104601.0,OBE,2017-09-01,FEMME,AGE DE 31-45,8 ans+,0-4 mois


In [63]:
CLIENT.isnull().sum().sum()

0

In [64]:
CLIENT.shape

(203760, 7)

In [65]:
CLIENTS = "C:\\Users\\belga\\OneDrive\\Bureau\\EMBA Big Data\\Module Datascience\\Projet Data Science S2 2024\\CLIENTS_NET.pkl"
CLIENT.to_pickle(CLIENTS)